In [7]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [8]:
scenarios_df = CSV.read("load_profiles.csv", DataFrame)
n_scenarios = size(scenarios_df, 2)
in_sample_n = 125

125

In [9]:

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

@variable(model, cap_bid)
@variable(model, beta <= 0)
@variable(model, theta[1:60, 1:in_sample_n])

@constraint(model, cap_bid >= 0)

for scenario in 1:in_sample_n
    for minute in 1:60
        @constraint(model, cap_bid - scenarios_df[minute, scenario] <= theta[minute, scenario])
        @constraint(model, beta <= theta[minute, scenario])
    end
end
@constraint(model, (1/(60*in_sample_n))*sum(theta[minute, scenario] for scenario in 1:in_sample_n, minute in 1:60) <= 0.9*beta) 

# Define the objective function
@objective(model, Max, cap_bid)

# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    #println("Optimal solution found")
    println("Optimal solution cap_bid: ", value(cap_bid))
elseif status == MOI.INFEASIBLE
    println("No feasible solution found")
else
    println("Solver status: ", status)
end

Optimal solution cap_bid: 285.86367935399085
